In [1]:
import pandas as pd
import os
import re
import json
from tableau_api_lib import TableauServerConnection
from tableau_api_lib.utils import querying, flatten_dict_column
from typing import List

In [2]:
root_path = r'C:\Users\NITINS\OneDrive - Capgemini\CAPGEMINI\PROJECT\GEN AI\report-usecase\tableau\tableau-auth\pat\890Portal'
print(root_path)

C:\Users\NITINS\OneDrive - Capgemini\CAPGEMINI\PROJECT\GEN AI\report-usecase\tableau\tableau-auth\pat\890Portal


In [3]:
sitename = '890Portal'

In [17]:
config = {
    "tableau-server" : {
        "server" : 'http://10.100.252.218/',
        "api_version" : "3.19",
        "personal_access_token_name" : "rest_token",
        "personal_access_token_secret" : "zL6zdQRgS9WQYH24aGR4JQ==:7yDTI0BCA6XlNxOm24YDN6S3B6673Nyn",
        "site_name" : "890Portal",
        "site_url" : "890Portal"
    }
}

In [20]:
conn = TableauServerConnection(config_json=config,env="tableau-server")

In [21]:
response = conn.sign_in()

In [22]:
response

<Response [200]>

In [23]:
def create_view_metadata(sitename, root_path):
    views_csv = os.path.join(root_path, f'{sitename}_Views.csv')
    workbooks_csv = os.path.join(root_path, f'{sitename}_Workbooks.csv')
    metadata_csv = os.path.join(root_path, f'{sitename}_View_Metadata.csv')
    unextracted = []
    extracted = []
    print(f'View : {views_csv} ; Workbook : {workbooks_csv}')

    # Load the Views CSV
    views_df = pd.read_csv(views_csv)
    workbooks_df = pd.read_csv(workbooks_csv)

    # Create a directory for saving images if it doesn't exist
    os.makedirs(os.path.join(root_path, 'Dashboard_images'), exist_ok=True)
    
    print(f'Processing view:')
    filtered_views = views_df[views_df['sheetType'] == 'view'].dropna(subset=['name', 'workbook_name'])
    metadata_df = pd.DataFrame()
    metadata_df['View name'] = filtered_views["name"]
    metadata_df['Workbook name'] = filtered_views["workbook_name"]
    metadata_df['Site name'] = [sitename] * len(filtered_views)
    metadata_df['View alias'] = filtered_views["viewUrlName"]

    filtered_views['Workbook url'] = filtered_views['contentUrl'].apply(lambda x: x.split('/')[0])
    metadata_df['Workbook url'] = filtered_views['Workbook url'].values

    # Extract Project name from Workbooks CSV
    project_names = []
    for workbook_name in filtered_views['workbook_name']:
        project_json = workbooks_df[workbooks_df['name'] == workbook_name]['project'].values
        if len(project_json) > 0:
            project_dict = json.loads(project_json[0].replace("'", "\""))  # Ensure correct JSON format
            project_names.append(project_dict['name'])
        else:
            project_names.append('')  # Handle cases where there is no matching workbook name

    metadata_df['Project name'] = project_names

    column_names = []
    for view_name, view_alias, project_name, workbook_url in zip(metadata_df['View name'], metadata_df['View alias'], metadata_df['Project name'], metadata_df['Workbook url']):
        if pd.notna(view_name) and pd.notna(project_name) and pd.notna(workbook_url):
            file_path = os.path.join(root_path, project_name, workbook_url, f'{view_name}.csv')
            if os.path.exists(file_path):
                try:
                    view_data_df = pd.read_csv(file_path)
                    columns = [col for col in view_data_df.columns if col != 'Unnamed: 0']
                    column_names.append(', '.join(columns))
                except Exception as e:
                    print(f'Error reading {file_path}: {e}')
                    column_names.append('')  # Handle cases where the file cannot be read
            else:
                alias_file_path = os.path.join(root_path, project_name, workbook_url, f'{view_alias}.csv')
                print(f'The alias found for the file : {alias_file_path}')
                if os.path.exists(alias_file_path):
                    try:
                        view_data_df = pd.read_csv(alias_file_path)
                        columns = [col for col in view_data_df.columns if col != 'Unnamed: 0']
                        column_names.append(', '.join(columns))
                    except Exception as e:
                        print(f'Error reading {alias_file_path}: {e}')
                        column_names.append('')  # Handle cases where the file cannot be read
                else:
                    print(f'Files do not exist: {file_path} and {alias_file_path}')
                    column_names.append('')  # Handle cases where neither file exists
        else:
            print(f'Invalid data for view: {view_name}, project: {project_name}, workbook: {workbook_url}')
            column_names.append('')  # Handle cases with NaN values

    metadata_df['Column name'] = column_names

    print(f'Metadata df : {metadata_df}')
    metadata_df.to_csv(metadata_csv, index=False)
        
    # Process dashboards
    print(f'Processing dashboard:')
    portal_dashdf = views_df[views_df['sheetType'] == 'dashboard']
    portal_dashdf = portal_dashdf[['name', 'id', 'contentUrl', 'sheetType', 'viewUrlName', 'workbook_name', 'workbook_id']]
    portal_dashdf.to_csv(os.path.join(root_path, '890Portal_Dashboards.csv'))

    for index, row in portal_dashdf.iterrows():
        dashid = row['id']
        print(f"Dashid : {dashid}")
        name = row['name']
        view_url_name = row['viewUrlName']
        workbook_name = row['workbook_name']

        # Check for special characters in the 'name' and replace the filename if necessary
        if re.search(r'[\\/]', name):
            filename = f'{workbook_name}_{view_url_name}' + '.png'
        else:
            filename = f'{workbook_name}_{name}' + '.png'

        print(f"Trying to extract the image for {dashid}")

        try:
            view_png = conn.query_view_image(view_id=dashid)

            # Save the image with the appropriate filename
            with open(os.path.join(root_path, 'Dashboard_images', filename), 'wb') as v:
                v.write(view_png.content)
            print(f'The Dashboard image for {filename} is extracted')
            extracted.append(filename)
        except Exception as e:
            print(f'The Dashboard image for {filename} could not be extracted: {e}')
            unextracted.append(filename)

    print(f'The dashboards for which the image could not be extracted: {unextracted}')
    print(f'The dashboards for which the image that are extracted: {extracted}')


In [24]:
create_view_metadata(sitename, root_path)

View : C:\Users\NITINS\OneDrive - Capgemini\CAPGEMINI\PROJECT\GEN AI\report-usecase\tableau\tableau-auth\pat\890Portal\890Portal_Views.csv ; Workbook : C:\Users\NITINS\OneDrive - Capgemini\CAPGEMINI\PROJECT\GEN AI\report-usecase\tableau\tableau-auth\pat\890Portal\890Portal_Workbooks.csv
Processing view:
The alias found for the file : C:\Users\NITINS\OneDrive - Capgemini\CAPGEMINI\PROJECT\GEN AI\report-usecase\tableau\tableau-auth\pat\890Portal\Default\ConsolidatedCookbook\RuralUrbanDistribution2019.csv
Metadata df :                                View name          Workbook name  Site name  \
8         Cycle Times on Insurance Claim  Consolidated Cookbook  890Portal   
9        Rural/Urban Distribution (2019)  Consolidated Cookbook  890Portal   
10        Promoters vs Detractors (2019)  Consolidated Cookbook  890Portal   
11                 Y-o-Y CSAT Comparison  Consolidated Cookbook  890Portal   
12                  Y-o-Y NPS Comparison  Consolidated Cookbook  890Portal   
14        

The Dashboard image for CMO_Dashboard-WIP_Reach.png is extracted
Dashid : c61b802a-8a5d-49fe-a6f9-91eb6e4ceea3
Trying to extract the image for c61b802a-8a5d-49fe-a6f9-91eb6e4ceea3
The Dashboard image for CMO_Dashboard-WIP_Promotion Effect.png is extracted
Dashid : 608163b1-d010-4514-9277-f693d8c64b26
Trying to extract the image for 608163b1-d010-4514-9277-f693d8c64b26
The Dashboard image for CMO_Dashboard-WIP_Conversion.png is extracted
Dashid : 25d485af-bfc4-4451-9fbb-661888b37ff7
Trying to extract the image for 25d485af-bfc4-4451-9fbb-661888b37ff7
The Dashboard image for CMO_Dashboard_Dashboard 1.png is extracted
Dashid : cbcc0b15-9c83-40a6-aea0-6768593ad506
Trying to extract the image for cbcc0b15-9c83-40a6-aea0-6768593ad506
The Dashboard image for CMO_Dashboard_Reach.png is extracted
Dashid : 2d15e4db-ea02-46ea-ae1c-4b23f43cc890
Trying to extract the image for 2d15e4db-ea02-46ea-ae1c-4b23f43cc890
The Dashboard image for CMO_Dashboard_Promotion Effect.png is extracted
Dashid : a31a6

The Dashboard image for Peek-a-Insights_Gross By Certification & Public Poles.png is extracted
Dashid : c183ebaf-2dd0-4d75-a091-f9a733277b2a
Trying to extract the image for c183ebaf-2dd0-4d75-a091-f9a733277b2a
The Dashboard image for Peek-a-Insights_Gross By IMDB Ratings.png is extracted
Dashid : f2320bbc-1fb8-4715-af99-4da5b3e3062a
Trying to extract the image for f2320bbc-1fb8-4715-af99-4da5b3e3062a
The Dashboard image for Invoice_Total Invoice.png is extracted
Dashid : 1fcf4db5-fe32-4952-b470-111b31034879
Trying to extract the image for 1fcf4db5-fe32-4952-b470-111b31034879
The Dashboard image for Invoice_Tax.png is extracted
Dashid : 22cc7cb5-aa96-41ee-b624-bc8da9e65bce
Trying to extract the image for 22cc7cb5-aa96-41ee-b624-bc8da9e65bce
The Dashboard image for Invoice_Gross.png is extracted
Dashid : d950cdd8-1471-4ae4-8cb5-2b05b79a0ade
Trying to extract the image for d950cdd8-1471-4ae4-8cb5-2b05b79a0ade
The Dashboard image for Backtest Asset Allocation Dashboard_Asset Allocation.png

In [25]:
conn.sign_out()

<Response [204]>

In [ ]:
def create_view_metadata1(sitename, root_path):
    views_csv = os.path.join(root_path, f'{sitename}_Views.csv')
    workbooks_csv = os.path.join(root_path, f'{sitename}_Workbooks.csv')
    metadata_csv = os.path.join(root_path, f'{sitename}_View_Metadata.csv')
    
    print(f'View : {views_csv} ; Workbook : {workbooks_csv}')
    
    # Load the Views CSV
    views_df = pd.read_csv(views_csv)
    workbooks_df = pd.read_csv(workbooks_csv)
    
    # Filter Views where sheetType is 'view'
    filtered_views = views_df[views_df['sheetType'] == 'view']
    
    # Extract workbook_name and view_name from contentUrl
    filtered_views['workbook_url'] = filtered_views["workbook_name"]
    filtered_views['workbook_name'] = filtered_views['contentUrl'].apply(lambda x: x.split('/')[0])
    filtered_views['view_name'] = filtered_views['contentUrl'].apply(lambda x: x.split('/')[-1])
    
    # Drop rows with NaN values in critical columns
    filtered_views = filtered_views.dropna(subset=['view_name', 'workbook_name'])
    
    print(f'Filtered view names: {filtered_views["view_name"].tolist()}')
    print(f'Filtered workbook names: {filtered_views["workbook_name"].tolist()}')
    print(f'Filtered workbook url: {filtered_views["workbook_url"].tolist()}')
    
    
    # Initialize the metadata dataframe
    metadata_df = pd.DataFrame()
    metadata_df['Site name'] = [sitename] * len(filtered_views)
    metadata_df['View name'] = filtered_views['view_name'].values
    metadata_df['Workbook name'] = filtered_views['workbook_name'].values
    metadata_df['Workbook url'] = filtered_views['workbook_url']
    
    # Extract Project name from Workbooks CSV
    project_names = []
    for workbook_name in filtered_views['workbook_name']:
        project_json = workbooks_df[workbooks_df['name'] == workbook_name]['project'].values
        if len(project_json) > 0:
            project_dict = json.loads(project_json[0].replace("'", "\""))  # Ensure correct JSON format
            project_names.append(project_dict['name'])
        else:
            project_names.append('')  # Handle cases where there is no matching workbook name
    
    metadata_df['Project name'] = project_names
    
    # Extract Column names by traversing the file structure
    column_names = []
    for view_name, project_name, workbook_name in zip(metadata_df['View name'], metadata_df['Project name'], metadata_df['Workbook url']):
        if pd.notna(view_name) and pd.notna(project_name) and pd.notna(workbook_name):
            file_path = os.path.join(root_path, project_name, workbook_name, f'{view_name}.csv')
            print(f'Trying to read file: {file_path}')
            if os.path.exists(file_path):
                try:
                    view_data_df = pd.read_csv(file_path)
                    print(f'Columns in {file_path}: {view_data_df.columns.tolist()}')
                    column_names.append(', '.join(view_data_df.columns))
                except Exception as e:
                    print(f'Error reading {file_path}: {e}')
                    column_names.append('')  # Handle cases where the file cannot be read
            else:
                print(f'File does not exist: {file_path}')
                column_names.append('')  # Handle cases where the file does not exist
        else:
            print(f'Invalid data for view: {view_name}, project: {project_name}, workbook: {workbook_name}')
            column_names.append('')  # Handle cases with NaN values
    
    metadata_df['Column name'] = column_names
    
    print(f'Metadata dataframe:\n{metadata_df}')

In [ ]:
create_view_metadata1(sitename, root_path)